In [1]:
import requests
import pandas as pd

# INPUT PARAMETERS
LeagueName = "Settlers"   # Put in the name of the league you want to check for
SellValuePercentage = 100 # Set to whatever you put the "Asking Price of POE Ninja at" (in Percent)




# CODE
SellValue = SellValuePercentage / 100

# Function to get items containing "Deafening" in their names
def get_deafening_items():
    url = f"https://poe.ninja/api/data/itemoverview?league={LeagueName}&type=Essence"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        filtered_items = [{'name': item['name'], 'chaosValue': item['chaosValue']} for item in data['lines'] if "Deafening" in item['name']]
        return filtered_items
    else:
        print(f"Failed to retrieve data from {url}: {response.status_code}")
        return []

# Function to get the value of "Primal Crystallised Lifeforce"
def get_primal_lifeforce_value():
    url = f"https://poe.ninja/api/data/currencyoverview?league={LeagueName}&type=Currency"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        for item in data['lines']:
            if item['currencyTypeName'] == "Primal Crystallised Lifeforce":
                return item['pay']['value']
        print("Primal Crystallised Lifeforce not found in the data.")
        return None
    else:
        print(f"Failed to retrieve data from {url}: {response.status_code}")
        return None

# Get deafening items
deafening_items = get_deafening_items()


if deafening_items:
    df = pd.DataFrame(deafening_items)
else:
    print("No Deafening items found.")

# Get primal lifeforce value
primal_lifeforce_value = get_primal_lifeforce_value()

# Calculate Reroll Cost
if primal_lifeforce_value is not None:
    Reroll_Cost = 30 / primal_lifeforce_value
    print("Reroll Cost per Essence:\t%.3f Chaos" %Reroll_Cost)

    # Lists to store essences worth rerolling and not worth rerolling
    worth_reroll_list = []
    not_worth_reroll_list = []

    mean_value = df['chaosValue'].mean()
    total_value = df['chaosValue'].sum()
    num_essences = len(df)
    print("Average Value of Essences:\t%.3f Chaos" %(mean_value))


    # Iterate through the entries of df
    for index, row in df.iterrows():
        value_other = total_value - row['chaosValue']
        average_sell_value = value_other / (num_essences - 1) if num_essences > 0 else 0

        # Calculate the potential value of rerolling
        expected_value = average_sell_value - row['chaosValue'] - Reroll_Cost

        # Add to appropriate list
        if expected_value > 0:
            worth_reroll_list.append((row['name'], row['chaosValue'], expected_value))
        else:
            not_worth_reroll_list.append((row['name'], row['chaosValue'], expected_value))

    # Print the results
    print("\nEssences worth rerolling:")
    for name, value, EV in worth_reroll_list:
        print("%s:\t\tValue: %.2f Chaos\tEV: %.2f Chaos" %(name, value, EV))

    print("\nEssences not worth rerolling:")
    for name, value, EV in not_worth_reroll_list:
        print("%s:\t\tValue: %.2f Chaos\tEV: %.2f Chaos" %(name, value, EV))
else:
    print("Cannot calculate Reroll Cost without Primal Crystallised Lifeforce value.")


Reroll Cost per Essence:	0.441 Chaos
Average Value of Essences:	2.82 Chaos

Essences worth rerolling:
Deafening Essence of Anguish:		Value: 2.00 Chaos	EV: 0.42 Chaos
Deafening Essence of Doubt:		Value: 2.00 Chaos	EV: 0.42 Chaos
Deafening Essence of Dread:		Value: 2.00 Chaos	EV: 0.42 Chaos
Deafening Essence of Fear:		Value: 2.00 Chaos	EV: 0.42 Chaos
Deafening Essence of Greed:		Value: 2.00 Chaos	EV: 0.42 Chaos
Deafening Essence of Sorrow:		Value: 2.00 Chaos	EV: 0.42 Chaos
Deafening Essence of Suffering:		Value: 2.00 Chaos	EV: 0.42 Chaos
Deafening Essence of Torment:		Value: 2.00 Chaos	EV: 0.42 Chaos

Essences not worth rerolling:
Deafening Essence of Contempt:		Value: 6.00 Chaos	EV: -3.79 Chaos
Deafening Essence of Envy:		Value: 4.73 Chaos	EV: -2.45 Chaos
Deafening Essence of Zeal:		Value: 3.98 Chaos	EV: -1.66 Chaos
Deafening Essence of Misery:		Value: 3.00 Chaos	EV: -0.63 Chaos
Deafening Essence of Scorn:		Value: 3.00 Chaos	EV: -0.63 Chaos
Deafening Essence of Wrath:		Value: 2.88 Chaos